In [ ]:
import tensorflow as tf
import numpy as np
import tf2onnx
import onnx

from lhrNetUtils.data_loader import lhrNetDataLoader
from lhrNetUtils import Config

config = Config("config.json")

In [ ]:
n_states = len(config.get_states())

testDataLoader = lhrNetDataLoader("data/test_details.csv",config.get_y_length(),config.get_x_length(),n_states,8)
trainDataLoader = lhrNetDataLoader("data/train_details.csv",config.get_y_length(),config.get_x_length(),n_states,8)

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(config.get_y_length(),config.get_x_length())),
    tf.keras.layers.Dense(128,activation="relu"),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(64,activation="relu"),
    tf.keras.layers.Dense(n_states,activation="softmax")
])

loss_fn = tf.keras.losses.CategoricalCrossentropy()
model.compile(optimizer="adam",loss=loss_fn,metrics=["accuracy"])

In [ ]:
model.fit(trainDataLoader,epochs=25)

In [ ]:
result = model.evaluate(testDataLoader)
dict(zip(model.metrics_names, result))

In [ ]:
np.set_printoptions(suppress=True)

In [ ]:
model.save("data/model.keras")

In [ ]:


model.output_names=["output"]
spec = (tf.TensorSpec((None, 32, 64), tf.float32, name="input"),)
onnx_model,_ = tf2onnx.convert.from_keras(model,input_signature=spec)

In [ ]:
onnx.save(onnx_model,"data/model.onnx")

In [ ]:
print(model.input_shape)
print([node.name for node in onnx_model.graph.output])